In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {
    "id": "5MMHg3FkSaTI"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Requirement already satisfied: bitsandbytes in /usr/local/lib/python3.10/dist-packages (0.43.1)\n",
      "Requirement already satisfied: git+https://github.com/huggingface/transformers.git in /usr/local/lib/python3.10/dist-packages (4.41.0.dev0)\n",
      "Requirement already satisfied: git+https://github.com/huggingface/peft.git in /usr/local/lib/python3.10/dist-packages (0.11.0.dev0)\n",
      "Requirement already satisfied: git+https://github.com/huggingface/accelerate.git in /usr/local/lib/python3.10/dist-packages (0.30.0.dev0)\n",
      "Requirement already satisfied: datasets in /usr/local/lib/python3.10/dist-packages (2.19.0)\n",
      "Requirement already satisfied: scipy in /usr/local/lib/python3.10/dist-packages (1.13.0)\n",
      "Requirement already satisfied: ipywidgets in /usr/local/lib/python3.10/dist-packages (7.7.2)\n",
      "Requirement already satisfied: matplotlib in /usr/local/lib/python3.10/dist-packages (3.7.1)\n",
      "Requirement already satisfied: gradio in /usr/local/lib/python3.10/dist-packages (4.29.0)\n",
      "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.\n",
      "gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2024.5.0 which is incompatible.\n"
     ]
    }
   ],
   "source": [
    "# System setup (install necessary packages only once)\n",
    "!pip install -q -U bitsandbytes\n",
    "!pip install -q -U git+https://github.com/huggingface/transformers.git\n",
    "!pip install -q -U git+https://github.com/huggingface/peft.git\n",
    "!pip install -q -U git+https://github.com/huggingface/accelerate.git\n",
    "!pip install -q -U datasets scipy ipywidgets matplotlib gradio"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {
    "id": "gq4Rl6c8ftuU"
   },
   "outputs": [],
   "source": [
    "import os\n",
    "import torch\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "from datasets import load_from_disk\n",
    "from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig\n",
    "from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {
    "id": "ZX8rppbCv7Sr"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Using device: cuda\n"
     ]
    }
   ],
   "source": [
    " # ✅ Step 2: Set Device\n",
    "device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")\n",
    "print(f'Using device: {device}')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {
    "id": "HRN332IrmBZu"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Mounted at /content/drive\n"
     ]
    }
   ],
   "source": [
    "from google.colab import drive\n",
    "drive.mount('/content/drive',force_remount=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {
    "id": "B82yem9nmBWO"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Extraction done!\n",
      "Extracted files:\n",
      "test\n",
      "train\n",
      "dataset_dict.json\n"
     ]
    }
   ],
   "source": [
    "import zipfile\n",
    "import os\n",
    "\n",
    "zip_path = '/content/drive/My Drive/dataa/train_test_data.zip'\n",
    "extract_path = '/content/train_test_data'  # You can change this folder if you want\n",
    "\n",
    "with zipfile.ZipFile(zip_path, 'r') as zip_ref:\n",
    "    zip_ref.extractall(extract_path)\n",
    "\n",
    "print(\"Extraction done!\")\n",
    "\n",
    "# Step 3: List extracted files to verify\n",
    "extracted_files = os.listdir(extract_path)\n",
    "print(\"Extracted files:\")\n",
    "for file in extracted_files:\n",
    "    print(file)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {
    "id": "uje5AVTjqzAV"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Train example: {'instruction': \"If you are a doctor, please answer the medical questions based on the patient's description.\", 'input': 'My grand daughter just came from her OB and they said that she definitely was pregnant, but now it is detaching from the uterus. She has slight cramping; nothing major, no bleeding. She was given a prescription for suppositories to induce the D&C affect. Should she take these without any other symptoms present at this time?', 'output': \"Respected user, thanks for using Chat Doctor.come have evaluated your query thoroughly.* Cramps during pregnancy may be due to - non-specific cause - uterine expansion via growing fetus - placental separation - fetal movements related - other abdominal causes * Only pain is not the criteria for induction of D&C effect, there are many parameters gone doctor evaluate and then decide for the prescription.* I strongly recommend following strictly your doctor's advise only. Hope to clarify your query. Welcome for further assistance. Thanks for using Chat Doctor .com & giving opportunity to assistWishing her fine recovery. Regards.\"}\n",
      "Eval example: {'instruction': \"If you are a doctor, please answer the medical questions based on the patient's description.\", 'input': 'hello doctor, i am 21 years old. i am about 5ft 5 , i weigh 60 kg, i am a female, i am of the blood group AS. i usually have abdominal pain just a little above my pubic area and its usually painful and affects my standing position. whenever this happens, i feel very bloated as my stomach becomes bigger, when this happens, i go to the toilet and after easing my self and letting gas out i feel better. but the problem is, it happens frequently. however, some months back, i had a scan which showed a bulky non gravid anteverted uterus with heterogeneous echo pattern. i have always had the pain even before i had the scan and after also. i also have menstrual cramps during my cycle which lasts at times for a day or two. before the scan, i had a medical abortion in which the pregnancy was about 4 weeks and 3 days old. pls doctor, i am confused and dont know what to do. i need your advice. what is the cause of this abdominal pain? does the M A have any thing to do with my scan result? what can i do ? Thank you', 'output': 'Hi;welcome to Chat Doctor The cause of your abdominal pain above the pubic region can be due to infection in pelvic region;may be due to bulky uterus;infection of intestine that is lower bowel/rectum but as the pain gets relieved it could be due to census if the pain comes and goes on its own after some time and is better after passing stools or gas. This sort of pain if you have it could be due to infection of intestine for which you need to take antibiotics. The bulky uterus can be as a result of MA, but there can be other reasons that need to be examined, so please consult a Gynecologist for examination and treatment. Thanks'}\n"
     ]
    }
   ],
   "source": [
    "from datasets import load_from_disk\n",
    "\n",
    "# Load the dataset\n",
    "dataset = load_from_disk(\"/content/train_test_data\")\n",
    "\n",
    "# Split into training and evaluation sets\n",
    "train_dataset = dataset['train']\n",
    "eval_dataset = dataset['test']\n",
    "\n",
    "# Optional: peek at a few examples\n",
    "print(\"Train example:\", train_dataset[0])\n",
    "print(\"Eval example:\", eval_dataset[0])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {
    "id": "XAN3G7VHrsC8"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "### The following is a doctor's opinion on a patient's query:\n",
      "### Patient query: My grand daughter just came from her OB and they said that she definitely was pregnant, but now it is detaching from the uterus. She has slight cramping; nothing major, no bleeding. She was given a prescription for suppositories to induce the D&C affect. Should she take these without any other symptoms present at this time?\n",
      "### Doctor opinion: Respected user, thanks for using Chat Doctor.come have evaluated your query thoroughly.* Cramps during pregnancy may be due to - non-specific cause - uterine expansion via growing fetus - placental separation - fetal movements related - other abdominal causes * Only pain is not the criteria for induction of D&C effect, there are many parameters gone doctor evaluate and then decide for the prescription.* I strongly recommend following strictly your doctor's advise only. Hope to clarify your query. Welcome for further assistance. Thanks for using Chat Doctor .com & giving opportunity to assistWishing her fine recovery. Regards.\n"
     ]
    }
   ],
   "source": [
    "def formatting_func(example):\n",
    "    return f\"### The following is a doctor's opinion on a patient's query:\\n### Patient query: {example['input']}\\n### Doctor opinion: {example['output']}\"\n",
    "\n",
    "print(formatting_func(train_dataset[0]))\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {
    "id": "ED_sh0y0s6qX"
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "The secret `HF_TOKEN` does not exist in your Colab secrets. To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session. You will be able to reuse this secret in all of your notebooks. Please note that authentication is recommended but still optional to access public models or datasets.\n"
     ]
    }
   ],
   "source": [
    "base_model_id = \"tiiuae/falcon-7b-instruct\"\n",
    "\n",
    "bnb_config = BitsAndBytesConfig(\n",
    "    load_in_4bit=True,\n",
    "    bnb_4bit_use_double_quant=True,\n",
    "    bnb_4bit_quant_type=\"nf4\",\n",
    "    bnb_4bit_compute_dtype=torch.bfloat16\n",
    ")\n",
    "\n",
    "model = AutoModelForCausalLM.from_pretrained(\n",
    "    base_model_id,\n",
    "    quantization_config=bnb_config,\n",
    "    device_map=\"auto\"\n",
    ")\n",
    "\n",
    "tokenizer = AutoTokenizer.from_pretrained(base_model_id)\n",
    "tokenizer.pad_token = tokenizer.eos_token"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {
    "id": "YcpgKjzjyElg"
   },
   "outputs": [],
   "source": [
    "max_length = 512\n",
    "\n",
    "def generate_and_tokenize_prompt(example):\n",
    "    tokenized = tokenizer(\n",
    "        formatting_func(example),\n",
    "        truncation=True,\n",
    "        max_length=max_length,\n",
    "        padding=\"max_length\"\n",
    "    )\n",
    "    tokenized[\"labels\"] = tokenized[\"input_ids\"].copy()\n",
    "    return tokenized\n",
    "\n",
    "tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)\n",
    "tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {
    "id": "h2h4iF7X0gEn"
   },
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAkQAAAHHCAYAAABeLEexAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bA PeAAAAAElFTkSuQmCC",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "def plot_data_lengths(train_data, val_data):\n",
    "    lengths = [len(x['input_ids']) for x in train_data] + [len(x['input_ids']) for x in val_data]\n",
    "    plt.hist(lengths, bins=20, color='skyblue')\n",
    "    plt.xlabel('Token Length')\n",
    "    plt.ylabel('Count')\n",
    "    plt.title('Token Length Distribution')\n",
    "    plt.show()\n",
    "\n",
    "plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {
    "id": "sMkGqK8d0gvz"
   },
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {
    "id": "zm6IUaOS0yqE"
   },
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {
    "id": "VmIDpVS_0ynZ"
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/usr/local/lib/python3.10/dist-packages/peft/mapping.py:105: UserWarning: You are using an old version of `peft` that is not compatible with `accelerate` >= 0.30.0. Please install `peft` >= 0.11.0 or `accelerate` < 0.30.0. It is recommended to use `pip install peft>=0.11.0`.\n",
      "  warnings.warn("
     ]
    }
   ],
   "source": [
    "model.gradient_checkpointing_enable()\n",
    "model = prepare_model_for_kbit_training(model)\n",
    "\n",
    "config = LoraConfig(\n",
    "    r=8,\n",
    "    lora_alpha=16,\n",
    "    target_modules=[\"query_key_value\", \"dense\", \"dense_h_to_4h\", \"dense_4h_to_h\"],  # Falcon-specific\n",
    "    bias=\"none\",\n",
    "    lora_dropout=0.05,\n",
    "    task_type=\"CAUSAL_LM\"\n",
    ")\n",
    "\n",
    "model = get_peft_model(model, config)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {
    "id": "iA45UIrR0yju"
   },
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "  <style>\n",
       "    /* Estilos para la tabla */\n",
       "    .custom-table {\n",
       "        border-collapse: collapse;\n",
       "        width: 100%;\n",
       "        font-family: Arial, sans-serif;\n",
       "    }\n",
       "    .custom-table th, .custom-table td {\n",
       "        border: 1px solid #ddd;\n",
       "        padding: 8px;\n",
       "        text-align: left;\n",
       "    }\n",
       "    .custom-table th {\n",
       "        background-color: #f2f2f2;\n",
       "    }\n",
       "    .custom-table tr:nth-child(even) {\n",
       "        background-color: #f9f9f9;\n",
       "    }\n",
       "    .custom-table tr:hover {\n",
       "        background-color: #f1f1f1;\n",
       "    }\n",
       "</style>\n",
       "<table class=\"custom-table\">\n",
       "  <thead>\n",
       " <tr style=\"text-align: left;\">\n",
       "      <th>Step</th>\n",
       "      <th>Training Loss</th>\n",
       "      <th>Validation Loss</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <td>10</td>\n",
       "      <td>2.657800</td>\n",
       "      <td>2.619276</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <td>20</td>\n",
       "      <td>2.558700</td>\n",
       "      <td>2.573808</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "             Step  Training Loss  Validation Loss\n",
       "0  10       2.657800         2.619276\n",
       "1  20       2.558700         2.573808"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling\n",
    "\n",
    "training_args = TrainingArguments(\n",
    "    output_dir=\"./falcon-patient-chatbot\",\n",
    "    per_device_train_batch_size=4,\n",
    "    gradient_accumulation_steps=2,\n",
    "    eval_strategy=\"steps\",\n",
    "    eval_steps=10,\n",
    "    save_steps=10,\n",
    "    logging_steps=5,\n",
    "    learning_rate=2e-4,\n",
    "    max_steps=20,\n",
    "    save_total_limit=1,\n",
    "    report_to=\"none\",\n",
    "    logging_dir=\"./logs\",\n",
    "    optim=\"paged_adamw_8bit\"\n",
    ")\n",
    "\n",
    "trainer = Trainer(\n",
    "    model=model,\n",
    "    args=training_args,\n",
    "    train_dataset=tokenized_train_dataset,\n",
    "    eval_dataset=tokenized_val_dataset,\n",
    "    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),\n",
    ")\n",
    "\n",
    "model.config.use_cache = False\n",
    "trainer.train()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {
    "id": "ax_-wHALdJua"
   },
   "outputs": [],
   "source": [
    "from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig\n",
    "from peft import PeftModel, PeftConfig\n",
    "import torch\n",
    "\n",
    "# === Step 1: Define model + quantization configs ===\n",
    "base_model_id = \"tiiuae/falcon-7b-instruct\"\n",
    "\n",
    "bnb_config = BitsAndBytesConfig(\n",
    "    load_in_4bit=True,\n",
    "    bnb_4bit_use_double_quant=True,\n",
    "    bnb_4bit_quant_type=\"nf4\",\n",
    "    bnb_4bit_compute_dtype=torch.bfloat16\n",
    ")\n",
    "\n",
    "# === Step 2: Load base model + tokenizer ===\n",
    "base_model = AutoModelForCausalLM.from_pretrained(\n",
    "    base_model_id,\n",
    "    quantization_config=bnb_config,\n",
    "    device_map=\"auto\"\n",
    ")\n",
    "\n",
    "tokenizer = AutoTokenizer.from_pretrained(base_model_id)\n",
    "\n",
    "# === Step 3: Load LoRA fine-tuned model from LOCAL checkpoint ===\n",
    "lora_checkpoint_path = \"/content/falcon-patient-chatbot/checkpoint-20\"\n",
    "\n",
    "ft_model = PeftModel.from_pretrained(base_model, lora_checkpoint_path)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {
    "id": "YMSrsJV5hSwF"
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.\n",
      "  warnings.warn(\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "✅ Merged model saved to './final_falcon_patient_model'\n"
     ]
    }
   ],
   "source": [
    "ft_model = ft_model.merge_and_unload()\n",
    "\n",
    "# === Step 5: Save the merged model to local directory ===\n",
    "ft_model.save_pretrained(\"./final_falcon_patient_model\")\n",
    "tokenizer.save_pretrained(\"./final_falcon_patient_model\")\n",
    "\n",
    "print(\"✅ Merged model saved to './final_falcon_patient_model'\")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {
    "id": "6HU3qyKMhSr3"
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "It is recommended to consult a doctor as soon as possible. They will be able to determine the cause of your symptoms and provide appropriate treatment. In the meantime, try to get some rest and stay hydrated.\n"
     ]
    }
   ],
   "source": [
    "from transformers import AutoModelForCausalLM, AutoTokenizer\n",
    "import torch\n",
    "\n",
    "# Load the merged model & tokenizer from your saved directory\n",
    "model_path = \"./final_falcon_patient_model\"\n",
    "tokenizer = AutoTokenizer.from_pretrained(model_path)\n",
    "model = AutoModelForCausalLM.from_pretrained(model_path, device_map=\"auto\")\n",
    "\n",
    "model.eval()\n",
    "\n",
    "def generate_response(query):\n",
    "    prompt = f\"### Patient query:\\n{query}\\n### Doctor opinion:\"\n",
    "    inputs = tokenizer(prompt, return_tensors=\"pt\").to(model.device)\n",
    "    with torch.no_grad():\n",
    "        outputs = model.generate(\n",
    "            **inputs,\n",
    "            max_new_tokens=125,\n",
    "            repetition_penalty=1.15,\n",
    "            do_sample=True,\n",
    "            temperature=0.7,\n",
    "            top_p=0.9,\n",
    "        )\n",
    "    response = tokenizer.decode(outputs[0], skip_special_tokens=True)\n",
    "    # Remove prompt part from the output\n",
    "    return response.replace(prompt, \"\").strip()\n",
    "\n",
    "# Test your model\n",
    "query = \"I have a headache and fever for two days, what should I do?\"\n",
    "print(generate_response(query))\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {
    "id": "xt0BC2ZBioVh"
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "The patient should avoid sitting or standing for long periods of time during the trip. It is important to take frequent breaks to stretch and walk around. The symptoms may be caused by the discomfort from sitting for long periods of time.\n"
     ]
    }
   ],
   "source": [
    "query = \"I have considerable lower back pain, also numbness in left buttocks and down left leg, girdling at the upper thigh.  MRI shows \\\"Small protrusiton of L3-4 interv. disc on left far laterally with annular fissuring fesulting in mild left neural foraminal narrowing with slight posterolateral displacement of the exiting l3 nerve root.\\\"  Other mild bulges L4-5 w/ fissuring, and mild buldge  L5-S1. 1) does this explain symptoms 2) I have a plane/car trip in 2 days lasting 8 hrs, then other travel.  Will this be harmful??\"\n",
    "print(generate_response(query))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {
    "id": "prnYi_dopyjH"
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py:322: UserWarning: The `gradio.Chatbot` component will be deprecated in a future version of Gradio. Use `gradio.MultimodalTextbox` instead.\n",
      "  warnings.warn(\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Colab notebook detected. To show errors in colab notebook, set debug=True in launch()\n",
      "Running on public URL: https://82424cf6dbd9fcf558.gradio.live\n",
      "This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal."
     ]
    }
   ],
   "source": [
    "import gradio as gr\n",
    "\n",
    "def doctor_bot(message, history):\n",
    "    response = generate_response(message)\n",
    "    return response\n",
    "\n",
    "gr.ChatInterface(\n",
    "    fn=doctor_bot, \n",
    "    title=\"🩺 AI Doctor Chatbot\",\n",
    "    chatbot=gr.Chatbot(height=300),\n",
    "    textbox=gr.Textbox(placeholder=\"Ask me a medical question...\", container=False, scale=7),\n",
    "    theme=\"soft\",\n",
    "    examples=[\n",
    "        [\"I have a headache and a fever. What should I do?\"],\n",
    "        [\"My child has a rash on their arm. What could it be?\"],\n",
    "        [\"What are the common symptoms of the flu?\"],\n",
    "    ],\n",
    "    cache_examples=False,\n",
    ").launch(share=True)\n"
   ]
  }
 ],
 "metadata": {
  "accelerator": "GPU",
  "colab": {
   "gpuType": "T4",
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "name": "python3"
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}